In [43]:
include("../src/Conduction.jl");
using Plots, LaTeXStrings, QuadGK, Roots;

semiconductor = Conduction.Semiconductor(
4.34e7, # decay constant of the assumed hydrogen-like localized state wave functions (cm^-1)
0.1 * 1.6e-19, # Mode effect of the phonons (J)
3e21, # intrinsic semiconductor's density (cm^-3)
3e21 * 0.001, # Doping states' density (cm^-3)
0.29 * 1.6e-19, # Energy to a vacant target site (J)
1e13, # Base electron jump rate
-0.62, # Fermi level (J)
0.1, # Intrinsic semiconductor's gaussian width (J)
0.1, # Doping states' gaussian width (J)
);

F = -5.3e4; # Field (V.cm^-1)
T = 300;

In [44]:
ratio = [0, 0.06, 0.09, 0.15];

p() = quadgk(
    r -> Conduction.DOS(semiconductor, r, T) * Conduction.F(semiconductor, r, T) * Conduction.k * T,
    -Inf,
    Inf
)[1];

function quantityCarrier(x)
    semiconductor.Ni = x;
    semiconductor.Nd = 0.001 * x;
    return p()
end

function find_multiple_zeros(x)
    density = (r -> find_zero(z -> quantityCarrier(z) - r, 1e20)).(x)
    return density
end
range = vcat([[i for i in 1:9] .* 10^i for i in 16:1:17]...);
densities = find_multiple_zeros(range);

In [ ]:
range_u = -15:1:15;
res_d = [];
res_u = [];

for i in densities
    res_d_partial= [];
    res_u_partial= [];
    semiconductor.Ni = x;
    semiconductor.Nd = 0.001 * x;
    for u in range_u:
        append!(res_d_partial, Conduction.D(semiconductor, u, T, F));
        append!(res_u_partial, Conduction.electronMobility(semiconductor, u, T, F));
    end

    append!(res_d, res_d_partial);
    append!(res_u, res_u_partial);
end